# Stellar Parameter Estimation Using astroPIPS

In this example notebook, we will go through an example of using the `astroPIPS` library to estimate the stellar parameters for RR Lyrae stars. As of version 0.3.0 of `astroPIPS`, we only have empirically derived relationships for stellar parameter estimation applying to RRab and RRc variables. 

In [1]:
# import astroPIPS

import PIPS

Now we need to import our variable star data
- Import data from a .dat data file
- Create a PIPS.photdata object which we call star

In [2]:
data = PIPS.data_readin_LPP('sample_data/002.dat',filter='V')
x, y, yerr = data
star = PIPS.photdata(data)

We now need to estimate the period by running the `get_period` method on our `photdata` object, we choose 5 fourier terms to fit the data with. 

In [3]:
period,err = star.get_period(Nterms=5,multiprocessing=True)

print("The period is " + str(period) + " +/- " + str(err) + " days")

/Users/jamessunseri/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/jamessunseri/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


The period is 0.5748866247400108 +/- 7.505536469181548e-06 days


If we try to start estimating stellar parameters before calculating the epoch, `astroPIPS` will not be able to properly calculate the stellar parameters. So we need to calculate the epoch offset.

In [4]:
star.get_epoch_offset()

print("The epoch offset is " + str(star.epoch_offset) + " days.")

The epoch offset is 0.4212382475572452 days.


Now we can calculate stellar parameters! Let's do it. I recommend checking out the methods available first before actually calculating parameters. To use a predicted model, Create a model object from a photdata object. Here we are using the Cacciari2005 model. This model was implemented using equations from Cacciari et al. 2005, citings to all originally derived models are in the docstrings for each parameter estimation method.

In [5]:
Model = PIPS.Cacciari2005(star)

In [6]:
#We can see all implemented methods available to us by using help(PIPS.Cacciari2005)
print("Here we can see all the different methods for parameter estimation \n\n")
help(PIPS.Cacciari2005)

Here we can see all the different methods for parameter estimation 


Help on class Cacciari2005 in module PIPS.class_StellarModels:

class Cacciari2005(StellarModels)
 |  Cacciari2005(star)
 |  
 |  Subclass for StellarModels corresponding to Cacciari's paper from 2005,
 |  this paper is commonly referenced in the literature for RR Lyrae Stellar
 |  Parameter relationships. We denote the original author's relationship's in
 |  the form of method doc strings
 |  
 |  Method resolution order:
 |      Cacciari2005
 |      StellarModels
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  calc_BV_0_ab(self)
 |      relationship empirically determined by Kovács & Walker (2001)
 |  
 |  calc_Fe_H_ab(self)
 |      relationship derived by Jurscik (1998)
 |  
 |  calc_Fe_H_c(self)
 |      relationship derived by Carretta & Gratton (1997)
 |  
 |  calc_M_v_ab(self)
 |      relationship derived by Kovács (2002)
 |  
 |  calc_M_v_c(self)
 |      relationship derived by Kovács (1998)
 |

Note, one can choose to pick certain stellar properties they want to calculate, by choosing from the list of methods. The typical naming structure is `calc_property_type()` or one can simply calculate all stellar properties with one line. 

In [7]:
#the sample data is of an RRab star
Model.calc_all_vals(star_type='RRab')

Similarly we can run `help()` again to see what properties have been calculated for our `Model` object. These properties are stored as object traits to the `Model` object.

In [8]:
help(Model.calc_all_vals)

Help on method calc_all_vals in module PIPS.class_StellarModels:

calc_all_vals(star_type) method of PIPS.class_StellarModels.Cacciari2005 instance
    "This function returns none. It should add several traits to your StellarModel Object:
    
    TRAITS
    
    self.Fe_H
    self.BV_0 (RRab only)
    self.log_T_eff
    self.M_v
    self.log_L
    self.log_M
    self.Fe_H_err
    self.BV_0_err (RRab only)
    self.log_T_eff_err
    self.M_v_err
    self.log_L_err
    self.log_M_err



Now we can access these traits:

In [9]:
print("The metallicity of this star is {0:0.5f} +/- {1:0.5f}".format(Model.Fe_H, Model.Fe_H_err))
print("The log(T_eff/[K]) of this star is {0:0.5f} +/- {1:0.5f}".format(Model.log_T_eff, Model.log_T_eff_err))
print("The log(L/[L_sun]) of this star is {0:0.5f} +/- {1:0.5f}".format(Model.log_L, Model.log_L_err))
print("The log(M/[M_sun]) of this star is {0:0.5f} +/- {1:0.5f}".format(Model.log_M, Model.log_M_err))

The metallicity of this star is -2.35203 +/- 0.12520
The log(T_eff/[K]) of this star is 3.80587 +/- 0.00147
The log(L/[L_sun]) of this star is 1.71139 +/- 0.00489
The log(M/[M_sun]) of this star is -0.16330 +/- 0.01260


There we have it. We have succesfully calculated the stellar properties of a RRab variable star. This can be done for an RRc star as well by modifying the `star_type` argument in the `Model.calc_all_vals()`method.